In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(os.path.join(os.getcwd(), ".."))
sys.path

['/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python310.zip',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10/lib-dynload',
 '',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10/site-packages',
 '/home/grads/blessyantony/anaconda3/envs/zoonosis/lib/python3.10/site-packages/PyQt5_sip-12.11.0-py3.10-linux-x86_64.egg',
 '/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments/../../..',
 '/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments/../..',
 '/home/grads/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/experiments/..']

In [2]:
import pandas as pd
import torchvision.datasets
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn
from torch.nn import Conv2d
import torch.nn.functional as F
import torch
import torchvision.transforms as transforms
import tqdm
from utils import utils, nn_utils
from utils.early_stopping import EarlyStopping
from sklearn.metrics import roc_curve, accuracy_score, f1_score, auc, precision_recall_curve
from statistics import mean

In [3]:
class CNN_2D_Model(nn.Module):
    def __init__(self, n_classes, N, n_filters, kernel_size, stride, img_size):
        super(CNN_2D_Model, self).__init__()
        # padding: same ensures the output has the same size as the input
        self.conv2d = Conv2d(in_channels=3,
                             out_channels=n_filters,
                             kernel_size=kernel_size,
                             stride=stride,
                             padding="same")
        self.conv2d_hidden = Conv2d(in_channels=n_filters,
                                    out_channels=n_filters,
                                    kernel_size=kernel_size,
                                    stride=stride,
                                    padding="same")
        # intermediate hidden layers (number = N-1): hidden_dim --> hidden_dim
        # N-1 because we already have one layer converting input_dim --> hidden_dim
        self.conv2d_hidden_layers = nn_utils.create_clones(self.conv2d_hidden, N - 1)
        self.linear = nn.Linear(img_size * img_size * n_filters, n_classes)

    def forward(self, X):
        X = F.relu(self.conv2d(X))
        for conv2d_hidden_layer in self.conv2d_hidden_layers:
            X = F.relu(conv2d_hidden_layer(X))

        # aggregate the embeddings from cnn
        # mean of the representations of all tokens
        self.cnn_emb = torch.flatten(X, 1)  # flatten all dimensions except batch
        y = self.linear(self.cnn_emb)
        return y
    
    
def get_cnn_model(model):
    cnn_model = CNN_2D_Model(n_classes=model["n_classes"],
                             N=model["depth"],
                             n_filters=model["n_filters"],
                             kernel_size=model["kernel_size"],
                             stride=model["stride"],
                             img_size=model["img_size"])

    print(cnn_model)
    print("Number of parameters = ", sum(p.numel() for p in cnn_model.parameters() if p.requires_grad))
    return cnn_model.to(nn_utils.get_device())

In [4]:
def run_model(model, train_dataset_loader, test_dataset_loader, loss, n_epochs, model_name, mode):
    tbw = SummaryWriter()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    max_lr = 1e-3
    lr_scheduler = OneCycleLR(
        optimizer=optimizer,
        max_lr=max_lr,
        epochs=n_epochs,
        steps_per_epoch=len(train_dataset_loader),
        pct_start=0.1,
        anneal_strategy='cos',
        div_factor=25.0,
        final_div_factor=10000.0)
    early_stopper = EarlyStopping(patience=10, min_delta=0)
    model.train_iter = 0
    model.test_iter = 0

    if mode == "train":
        # train the model only if set to train mode
        for e in range(n_epochs):
            model = run_epoch(model, train_dataset_loader, test_dataset_loader, criterion, optimizer,
                              lr_scheduler, early_stopper, tbw, model_name, e, max_lr)
            # check if early stopping condition was satisfied and stop accordingly
            if early_stopper.early_stop:
                print("Breaking off training loop due to early stop")
                break

    result_df, _ = evaluate_model(model, test_dataset_loader, criterion, tbw, model_name, epoch=None, max_lr=max_lr, log_loss=False), model
    return result_df


def run_epoch(model, train_dataset_loader, test_dataset_loader, criterion, optimizer, lr_scheduler, early_stopper, tbw, model_name,
              epoch, max_lr):
    # Training
    model.train()
    for _, record in enumerate(pbar := tqdm.tqdm(train_dataset_loader)):
        input, label = record
        input = input.to(nn_utils.get_device())
        label = label.to(nn_utils.get_device())
        optimizer.zero_grad()

        output = model(input)
        output = output.to(nn_utils.get_device())

        loss = criterion(output, label.long())
        loss.backward()

        optimizer.step()
        lr_scheduler.step()

        model.train_iter += 1
        curr_lr = lr_scheduler.get_last_lr()[0]
        train_loss = loss.item()
        tbw.add_scalar(f"{model_name}-max_lr{max_lr}/learning-rate", float(curr_lr), model.train_iter)
        tbw.add_scalar(f"{model_name}-max_lr{max_lr}/training-loss", float(train_loss), model.train_iter)
        pbar.set_description(
            f"{model_name}/training-loss = {float(train_loss)}, model.n_iter={model.train_iter}, epoch={epoch + 1}")

    # Testing
    _, val_loss = evaluate_model(model, test_dataset_loader, criterion, tbw, model_name, epoch, max_lr, log_loss=True)
    early_stopper(val_loss)
    return model


def evaluate_model(model, test_dataset_loader, criterion, tbw, model_name, epoch, max_lr, log_loss=False):
    with torch.no_grad():
        model.eval()

        results = []
        val_loss = []
        for _, record in enumerate(pbar := tqdm.tqdm(test_dataset_loader)):
            input, label = record
            input = input.to(nn_utils.get_device())
            label = label.to(nn_utils.get_device())
            output = model(input)  # b x n_classes
            output = output.to(nn_utils.get_device())

            loss = criterion(output, label.long())
            curr_val_loss = loss.item()
            model.test_iter += 1
            if log_loss:
                tbw.add_scalar(f"{model_name}-max_lr{max_lr}/validation-loss", float(curr_val_loss), model.test_iter)
                pbar.set_description(
                    f"{model_name}/validation-loss = {float(curr_val_loss)}, model.n_iter={model.test_iter}, epoch={epoch + 1}")
            val_loss.append(curr_val_loss)
            # to get probabilities of the output
            output = F.softmax(output, dim=-1)
            result_df = pd.DataFrame(output.cpu().numpy())
            result_df["y_true"] = label.cpu().numpy()
            results.append(result_df)
    return pd.concat(results, ignore_index=True), mean(val_loss)


def main():
    print("Main")
    transform = transforms.Compose([transforms.ToTensor()])
    batch_size = 64
    train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

    model = get_cnn_model({
        "n_classes": 10,
        "depth": 2,
        "n_filters": 3,
        "kernel_size": 3,
        "stride": 1,
        "img_size": 32
    })

    results_df, _ = run_model(model=model,
              train_dataset_loader=train_dataset_loader,
              test_dataset_loader=test_dataset_loader,
              loss="FocalLoss",
              n_epochs=50,
              model_name="CNN-CIFAR10",
              mode="train")
    return results_df

In [7]:
results_df = main()

Main
Files already downloaded and verified
Files already downloaded and verified
CNN_2D_Model(
  (conv2d): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2d_hidden): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (conv2d_hidden_layers): ModuleList(
    (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
  )
  (linear): Linear(in_features=3072, out_features=10, bias=True)
)
Number of parameters =  30982


CNN-CIFAR10/training-loss = 2.005497455596924, model.n_iter=782, epoch=1: 100%|██████████████████████████████████████████████████| 782/782 [00:11<00:00, 68.97it/s]
CNN-CIFAR10/validation-loss = 1.799963116645813, model.n_iter=157, epoch=1: 100%|████████████████████████████████████████████████| 157/157 [00:02<00:00, 77.25it/s]
CNN-CIFAR10/training-loss = 1.9968056678771973, model.n_iter=1564, epoch=2: 100%|████████████████████████████████████████████████| 782/782 [00:10<00:00, 71.11it/s]
CNN-CIFAR10/validation-loss = 1.5522332191467285, model.n_iter=314, epoch=2: 100%|███████████████████████████████████████████████| 157/157 [00:01<00:00, 85.55it/s]
CNN-CIFAR10/training-loss = 2.1254525184631348, model.n_iter=2346, epoch=3: 100%|████████████████████████████████████████████████| 782/782 [00:10<00:00, 74.53it/s]
CNN-CIFAR10/validation-loss = 1.616783618927002, model.n_iter=471, epoch=3: 100%|████████████████████████████████████████████████| 157/157 [00:01<00:00, 86.43it/s]
CNN-CIFAR10/trai

Early stopping counter: 1 / 10


CNN-CIFAR10/training-loss = 1.6810027360916138, model.n_iter=6256, epoch=8: 100%|████████████████████████████████████████████████| 782/782 [00:10<00:00, 75.31it/s]
CNN-CIFAR10/validation-loss = 1.4641616344451904, model.n_iter=1256, epoch=8: 100%|██████████████████████████████████████████████| 157/157 [00:02<00:00, 73.88it/s]
CNN-CIFAR10/training-loss = 1.8137720823287964, model.n_iter=7038, epoch=9: 100%|████████████████████████████████████████████████| 782/782 [00:10<00:00, 72.74it/s]
CNN-CIFAR10/validation-loss = 1.463633418083191, model.n_iter=1413, epoch=9: 100%|███████████████████████████████████████████████| 157/157 [00:01<00:00, 90.71it/s]
CNN-CIFAR10/training-loss = 1.3489887714385986, model.n_iter=7820, epoch=10: 100%|███████████████████████████████████████████████| 782/782 [00:10<00:00, 76.99it/s]
CNN-CIFAR10/validation-loss = 1.5918210744857788, model.n_iter=1570, epoch=10: 100%|█████████████████████████████████████████████| 157/157 [00:01<00:00, 88.54it/s]


Early stopping counter: 1 / 10


...

CNN-CIFAR10/training-loss = 1.5566688776016235, model.n_iter=21114, epoch=27: 100%|██████████████████████████████████████████████| 782/782 [00:10<00:00, 77.64it/s]
CNN-CIFAR10/validation-loss = 1.7382090091705322, model.n_iter=4239, epoch=27: 100%|█████████████████████████████████████████████| 157/157 [00:01<00:00, 86.93it/s]


Early stopping counter: 9 / 10


CNN-CIFAR10/training-loss = 1.5171399116516113, model.n_iter=21896, epoch=28: 100%|██████████████████████████████████████████████| 782/782 [00:10<00:00, 77.08it/s]
CNN-CIFAR10/validation-loss = 1.7025290727615356, model.n_iter=4396, epoch=28: 100%|█████████████████████████████████████████████| 157/157 [00:01<00:00, 88.25it/s]


Early stopping counter: 10 / 10
Early STOP: Early stopping threshold reached.
Breaking off training loop due to early stop


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 106.73it/s]


In [8]:
results_df

,0,1,2,3,4,5,6,7,8,9,y_true
0,0.003510,0.001911,0.019537,0.676966,2.538372e-02,6.445172e-02,0.180722,2.726518e-04,0.026997,0.000249,3
1,0.005806,0.683730,0.000002,0.000004,8.364514e-07,2.893021e-07,0.000002,8.526448e-07,0.286974,0.023479,8
2,0.635771,0.039265,0.012964,0.009985,9.364109e-04,1.381257e-02,0.000044,3.633298e-02,0.164494,0.086395,8
3,0.267318,0.042131,0.018912,0.003844,2.723092e-03,1.674157e-03,0.000259,6.129877e-03,0.627427,0.029581,0
4,0.000184,0.000003,0.031662,0.088187,7.064581e-01,6.908988e-02,0.083259,1.988525e-02,0.001066,0.000205,6
...,...,...,...,...,...,...,...,...,...,...,...
9995,0.314838,0.001216,0.057906,0.219833,3.552399e-02,1.082123e-01,0.000698,1.554670e-02,0.233642,0.012583,8
9996,0.001362,0.000007,0.036595,0.260279,7.905325e-02,1.538853e-01,0.464559,3.076355e-03,0.000775,0.000408,3
9997,0.000495,0.001814,0.069840,0.265926,2.403824e-02,5.148486e-01,0.027216,8.355962e-02,0.001051,0.011212,5
9998,0.075319,0.501914,0.010701,0.072903,1.140959e-01,1.122649e-01,0.062422,2.001339e-02,0.010820,0.019547,1


In [9]:
auprcs = []
for i in range(10):
    precision, recall, _ = precision_recall_curve(y_true=results_df["y_true"].values, probas_pred=results_df[i].values, pos_label=i)
    auprc = auc(recall, precision)
    print(f"AUPRC for class {i} = {auprc}")
    auprcs.append(auprc)

macro_auprc = mean(auprcs)
print(f"Macro AUPRC = {macro_auprc}")

AUPRC for class 0 = 0.6055322426403852
AUPRC for class 1 = 0.6568172694765939
AUPRC for class 2 = 0.34150691042283227
AUPRC for class 3 = 0.29919740934829064
AUPRC for class 4 = 0.3875204563843705
AUPRC for class 5 = 0.36611551888814065
AUPRC for class 6 = 0.5120589525176913
AUPRC for class 7 = 0.56449490195931
AUPRC for class 8 = 0.6171756019601253
AUPRC for class 9 = 0.5585778921044726
Macro AUPRC = 0.4908997155702212


In [5]:
class CNN_2D_MaxPool_Model(nn.Module):
    def __init__(self, n_classes, N, n_filters, kernel_size, stride, img_size):
        super(CNN_2D_MaxPool_Model, self).__init__()
        # padding: same ensures the output has the same size as the input
        self.conv2d_1 = Conv2d(in_channels=3,
                             out_channels=n_filters,
                             kernel_size=kernel_size,
                             stride=stride,
                             padding="same")
        self.maxpool = nn.MaxPool2d(2, 2)
        self.conv2d_2 = Conv2d(in_channels=n_filters,
                             out_channels=16,
                             kernel_size=kernel_size,
                             stride=stride,
                             padding="same")
        
        self.linear_1 = nn.Linear(int(16 * img_size/4 * img_size/4), 120)
        self.linear_2 = nn.Linear(120, 84)
        self.linear_3 = nn.Linear(84, 10)

    def forward(self, X):
        X = self.maxpool(F.relu(self.conv2d_1(X)))
        X = self.maxpool(F.relu(self.conv2d_2(X)))
        X = torch.flatten(X, 1) # flatten all dimensions except batch
        
        X = F.relu(self.linear_1(X))
        X = F.relu(self.linear_2(X))
        
        y = self.linear_3(X)
        return y
    
    
    
def get_cnn_maxpool_model(model):
    cnn_model = CNN_2D_MaxPool_Model(n_classes=model["n_classes"],
                             N=model["depth"],
                             n_filters=model["n_filters"],
                             kernel_size=model["kernel_size"],
                             stride=model["stride"],
                             img_size=model["img_size"])

    print(cnn_model)
    print("Number of parameters = ", sum(p.numel() for p in cnn_model.parameters() if p.requires_grad))
    return cnn_model.to(nn_utils.get_device())

In [6]:
def main_maxpool():
    print("Main Max Pool")
    transform = transforms.Compose([transforms.ToTensor()])
    batch_size = 64
    train_dataset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    test_dataset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

    train_dataset_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataset_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

    model = get_cnn_maxpool_model({
        "n_classes": 10,
        "depth": 2,
        "n_filters": 6,
        "kernel_size": 5,
        "stride": 1,
        "img_size": 32
    })

    results_df, _ = run_model(model=model,
              train_dataset_loader=train_dataset_loader,
              test_dataset_loader=test_dataset_loader,
              loss="FocalLoss",
              n_epochs=50,
              model_name="CNN-MaxPool-CIFAR10",
              mode="train")
    return results_df

In [7]:
results_df_1 = main_maxpool()

Main Max Pool
Files already downloaded and verified
Files already downloaded and verified
CNN_2D_MaxPool_Model(
  (conv2d_1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (linear_1): Linear(in_features=1024, out_features=120, bias=True)
  (linear_2): Linear(in_features=120, out_features=84, bias=True)
  (linear_3): Linear(in_features=84, out_features=10, bias=True)
)
Number of parameters =  136886


CNN-MaxPool-CIFAR10/training-loss = 1.5757601261138916, model.n_iter=782, epoch=1: 100%|█████████████████████████████████████████| 782/782 [00:13<00:00, 56.74it/s]
CNN-MaxPool-CIFAR10/validation-loss = 1.9495793581008911, model.n_iter=157, epoch=1: 100%|███████████████████████████████████████| 157/157 [00:01<00:00, 90.01it/s]
CNN-MaxPool-CIFAR10/training-loss = 1.621734380722046, model.n_iter=1564, epoch=2: 100%|█████████████████████████████████████████| 782/782 [00:10<00:00, 75.40it/s]
CNN-MaxPool-CIFAR10/validation-loss = 1.7317614555358887, model.n_iter=314, epoch=2: 100%|███████████████████████████████████████| 157/157 [00:01<00:00, 88.08it/s]
CNN-MaxPool-CIFAR10/training-loss = 1.4852652549743652, model.n_iter=2346, epoch=3: 100%|████████████████████████████████████████| 782/782 [00:10<00:00, 75.92it/s]
CNN-MaxPool-CIFAR10/validation-loss = 1.7497484683990479, model.n_iter=471, epoch=3: 100%|███████████████████████████████████████| 157/157 [00:01<00:00, 88.63it/s]
CNN-MaxPool-CIFA

Early stopping counter: 1 / 10


CNN-MaxPool-CIFAR10/training-loss = 0.7500495314598083, model.n_iter=10948, epoch=14: 100%|██████████████████████████████████████| 782/782 [00:10<00:00, 72.90it/s]
CNN-MaxPool-CIFAR10/validation-loss = 1.4235379695892334, model.n_iter=2198, epoch=14: 100%|█████████████████████████████████████| 157/157 [00:01<00:00, 82.97it/s]


Early stopping counter: 2 / 10


...

CNN-MaxPool-CIFAR10/training-loss = 0.6007446050643921, model.n_iter=16422, epoch=21: 100%|██████████████████████████████████████| 782/782 [00:11<00:00, 70.68it/s]
CNN-MaxPool-CIFAR10/validation-loss = 1.1327440738677979, model.n_iter=3297, epoch=21: 100%|█████████████████████████████████████| 157/157 [00:01<00:00, 81.19it/s]


Early stopping counter: 9 / 10


CNN-MaxPool-CIFAR10/training-loss = 0.6725125312805176, model.n_iter=17204, epoch=22: 100%|██████████████████████████████████████| 782/782 [00:10<00:00, 74.30it/s]
CNN-MaxPool-CIFAR10/validation-loss = 1.3234622478485107, model.n_iter=3454, epoch=22: 100%|█████████████████████████████████████| 157/157 [00:01<00:00, 85.88it/s]


Early stopping counter: 10 / 10
Early STOP: Early stopping threshold reached.
Breaking off training loop due to early stop


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:01<00:00, 104.84it/s]


In [8]:
results_df_1

,0,1,2,3,4,5,6,7,8,9,y_true
0,7.775296e-05,1.249700e-05,1.818604e-04,9.142494e-01,9.846402e-05,8.513452e-02,8.433924e-06,2.128123e-04,1.921865e-05,5.031317e-06,3
1,1.487698e-04,1.363557e-01,2.228991e-08,7.868332e-08,1.264316e-10,2.061656e-08,3.559669e-08,1.117268e-09,8.631737e-01,3.216984e-04,8
2,7.977743e-02,3.303924e-04,1.121504e-05,1.117484e-04,5.682932e-06,1.048974e-06,5.097469e-07,9.427195e-06,9.191543e-01,5.981280e-04,8
3,9.984623e-01,2.150432e-05,2.710773e-04,2.101128e-05,8.179437e-04,1.337411e-05,9.251529e-08,1.467364e-04,2.275555e-04,1.834625e-05,0
4,1.768320e-09,2.698723e-09,4.125910e-03,1.193616e-01,7.700981e-01,2.446160e-04,1.061489e-01,2.080914e-05,9.225453e-08,3.131278e-09,6
...,...,...,...,...,...,...,...,...,...,...,...
9995,2.455725e-02,3.384214e-03,1.898962e-05,1.680849e-01,4.522581e-06,5.822346e-01,4.518705e-06,3.813598e-05,2.216724e-01,4.756793e-07,8
9996,2.043237e-07,1.887062e-08,2.869339e-02,6.595091e-01,1.738558e-01,7.539368e-02,5.296532e-02,9.573013e-03,6.913088e-06,2.576378e-06,3
9997,2.980013e-05,1.475179e-10,1.809285e-02,2.045206e-01,7.095814e-04,7.716811e-01,1.948756e-04,4.770860e-03,2.195218e-08,2.891947e-07,5
9998,3.115217e-02,8.698265e-02,1.639027e-01,5.404172e-02,5.491359e-01,9.774116e-02,8.023535e-03,9.900077e-07,8.946112e-03,7.297987e-05,1


In [9]:
auprcs = []
for i in range(10):
    precision, recall, _ = precision_recall_curve(y_true=results_df_1["y_true"].values, probas_pred=results_df_1[i].values, pos_label=i)
    auprc = auc(recall, precision)
    print(f"AUPRC for class {i} = {auprc}")
    auprcs.append(auprc)

macro_auprc = mean(auprcs)
print(f"Macro AUPRC = {macro_auprc}")

AUPRC for class 0 = 0.7524650699727221
AUPRC for class 1 = 0.8273696235716458
AUPRC for class 2 = 0.5753369371556882
AUPRC for class 3 = 0.48172726322712794
AUPRC for class 4 = 0.6250637314604829
AUPRC for class 5 = 0.5901951602144044
AUPRC for class 6 = 0.7760028400180341
AUPRC for class 7 = 0.7822793126838635
AUPRC for class 8 = 0.8425138570007332
AUPRC for class 9 = 0.7733447249482502
Macro AUPRC = 0.7026298520252953
